In [1]:
#import pandas as pd
import numpy as np
from glob import glob
import os
import json
from PIL import Image
import albumentations as A
import random

/root/miniconda3/envs/kfold_amy_plaque1/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pip install -U albumentations

     |████████████████████████████████| 171 kB 31.2 MB/s eta 0:00:01
     |████████████████████████████████| 423 kB 106.7 MB/s eta 0:00:01
     |████████████████████████████████| 49.9 MB 103.8 MB/s eta 0:00:01
     |████████████████████████████████| 19.5 MB 35.3 MB/s eta 0:00:01
     |████████████████████████████████| 14.9 MB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 38.6 MB 31.0 MB/s eta 0:00:01
     |████████████████████████████████| 18.2 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 226 kB 123.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 58.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 40.6 MB/s eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.8.0
    Uninstalling typing-extensions-4.8.0:
      Successfully uninstalled typing-extensions-4.8.0
  Attempting uninstall: pillow
    Fo

In [2]:
path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/amy-def-mfg-jsons"

In [48]:
val_path = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/amy-def-mfg-test-jsons"

In [49]:
df = pd.DataFrame({"json_path":glob(os.path.join(path,"*.json"))})

In [50]:
val_df = pd.DataFrame({"json_path":glob(os.path.join(val_path,"*.json"))})

In [51]:
def get_count(l):
    with open(l) as f:
        data = json.load(f)
    plaque_dict = {'Cored': 0, 'Diffuse': 0, 'Coarse-Grained': 0,'CAA': 0}
    for tileId, ele in data.items():
        for region in ele:
            if "label" in region.keys():
                if "name" in region['label'].keys():
                    plaque_dict[region['label']["name"]]=plaque_dict[region['label']["name"]]+1
    return plaque_dict
                

In [5]:
for cls in ["Cored","Diffuse","Coarse-Grained","CAA"]:
    df[cls] = df["json_path"].apply(lambda l:get_count(l)[cls] )

In [20]:
for cls in ["Cored","Diffuse","Coarse-Grained","CAA"]:
    val_df[cls] = val_df["json_path"].apply(lambda l:get_count(l)[cls] )

In [11]:
df["file_name"] = df["json_path"].apply(lambda l: l.split("/")[-1].split(".")[0])

In [21]:
val_df["file_name"] = val_df["json_path"].apply(lambda l: l.split("/")[-1].split(".")[0])

In [12]:
df

,json_path,Cored,Diffuse,Coarse-Grained,CAA,file_name
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,21,26,3,0,XE07-047_1_AmyB_1
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,5,34,3,0,XE18-066_1_AmyB_1
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,13,18,4,8,XE07-064_1_AmyB_1
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,2,27,3,9,XE17-004_1_AmyB_1
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,23,22,14,0,XE15-039_1_AmyB_1
5,/gladstone/finkbeiner/steve/work/data/npsad_da...,1,20,0,2,XE19-037_1_AmyB_1
6,/gladstone/finkbeiner/steve/work/data/npsad_da...,10,13,10,9,XE17-039_1_AmyB_1
7,/gladstone/finkbeiner/steve/work/data/npsad_da...,27,97,5,0,XE07-048_1_AmyB_1
8,/gladstone/finkbeiner/steve/work/data/npsad_da...,26,27,6,6,XE15-022_1_AmyB_1
9,/gladstone/finkbeiner/steve/work/data/npsad_da...,6,20,10,6,XE10-045_1_AmyB_1


In [13]:
test_files =  [x.split("/")[-1] for x in glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/test","*"))]

In [28]:
val_files =  [x.split("/")[-1] for x in glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val","*"))]

In [29]:
val_files

['XE16-021_1_AmyB_1',
 'XE17-059_1_AmyB_1',
 'XE12-016_1_AmyB_1',
 'XE14-004_1_AmyB_1',
 'XE13-007_1_AmyB_1',
 'XE17-030_1_AmyB_1',
 'XE17-010_1_AmyB_1']

In [30]:
val_df["flag"] = val_df["file_name"].apply(lambda l: "val" if l in val_files else "train")

In [15]:
df["flag"] = df["file_name"].apply(lambda l: "test" if l in test_files else "train")

In [31]:
df1 = pd.concat([df,val_df], ignore_index=True)

In [32]:
df1.groupby(["flag"]).sum()

,Cored,Diffuse,Coarse-Grained,CAA
flag,,,,
test,26,85,22,15
train,246,393,131,66
val,49,69,62,13


In [33]:
df1

,json_path,Cored,Diffuse,Coarse-Grained,CAA,file_name,flag
0,/gladstone/finkbeiner/steve/work/data/npsad_da...,21,26,3,0,XE07-047_1_AmyB_1,train
1,/gladstone/finkbeiner/steve/work/data/npsad_da...,5,34,3,0,XE18-066_1_AmyB_1,test
2,/gladstone/finkbeiner/steve/work/data/npsad_da...,13,18,4,8,XE07-064_1_AmyB_1,train
3,/gladstone/finkbeiner/steve/work/data/npsad_da...,2,27,3,9,XE17-004_1_AmyB_1,train
4,/gladstone/finkbeiner/steve/work/data/npsad_da...,23,22,14,0,XE15-039_1_AmyB_1,train
5,/gladstone/finkbeiner/steve/work/data/npsad_da...,1,20,0,2,XE19-037_1_AmyB_1,test
6,/gladstone/finkbeiner/steve/work/data/npsad_da...,10,13,10,9,XE17-039_1_AmyB_1,train
7,/gladstone/finkbeiner/steve/work/data/npsad_da...,27,97,5,0,XE07-048_1_AmyB_1,train
8,/gladstone/finkbeiner/steve/work/data/npsad_da...,26,27,6,6,XE15-022_1_AmyB_1,train
9,/gladstone/finkbeiner/steve/work/data/npsad_da...,6,20,10,6,XE10-045_1_AmyB_1,train


## Data Augmentation

In [37]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/",'*.png'))

all_images.sort()
all_masks.sort()

cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [38]:
len(all_images)

471

In [39]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(213, 206, 106, 52)

In [40]:
transforms  = A.Compose([ A.VerticalFlip(p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.Blur(blur_limit=1,p=0.2),
                            A.RandomRotate90(),
                        ])

def get_randimages_dataug(total_imgs, image_filenames, label_filenames):
    image_filenames.sort()
    label_filenames.sort()
    random_image_file = []
    random_label_file = []
    for i in range(total_imgs):
        random.seed(i)
        random_image_file.append(random.choice(image_filenames))
        random.seed(i)
        random_label_file.append(random.choice(label_filenames))
    return [random_image_file, random_label_file]

def upsample_dataset(dataset_base_dir, random_img_filenames, rand_label_filenames, variations, transforms, dest_img_folder_name, dest_label_folder_name):
    i = 0
    aug_img_files = []
    aug_mask_files = []
    # random.seed(500)

    # Make dir where tha augmented file will reside
    aug_img_dir = os.path.join(dataset_base_dir, dest_img_folder_name)
    if not os.path.exists(aug_img_dir):
            os.makedirs(aug_img_dir)
            print("Augmented Directory '%s' created" %aug_img_dir)
    
    aug_mask_dir = os.path.join(dataset_base_dir, dest_label_folder_name)
    if not os.path.exists(aug_mask_dir):
            os.makedirs(aug_mask_dir)
            print("Augmented Directory '%s' created" %aug_mask_dir)

    print(os.listdir(dataset_base_dir))
    print("\nData Augmentation in Progress ...")
    total_imgs = len(random_img_filenames)
    
    for i in range(total_imgs):
        # load the image
        img = Image.open(random_img_filenames[i]).convert("RGB")
        img = np.array(img)
        mask = Image.open(rand_label_filenames[i]).convert('P')
        mask = np.array(mask)

        for j in range(variations):
                transformed = transforms(image=img, mask=mask)
                transformed_img = transformed["image"]
                transformed_img = Image.fromarray(transformed_img)

                #To rename the file with prefix A_
                filename = os.path.basename(random_img_filenames[i])
                filepath = os.path.dirname(random_img_filenames[i])

                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir,dest_img_folder_name,
                                        aug_file_name)
                transformed_img.save(new_file)
                aug_img_files.append(new_file)

                transformed_mask = transformed["mask"]
                transformed_mask = Image.fromarray(transformed_mask)

                #To rename the file with prefix A_
                filename = os.path.basename(rand_label_filenames[i])
                filepath = os.path.dirname(rand_label_filenames[i])
                aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                new_file = os.path.join(dataset_base_dir, dest_label_folder_name,
                                        aug_file_name)
                transformed_mask.save(new_file)
                aug_mask_files.append(new_file)
    return aug_img_files, aug_mask_files

In [41]:
aug_value = 100
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, cg_images, cg_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', '.DS_Store', 'images']

Data Augmentation in Progress ...


In [42]:
aug_value = 150
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, caa_images, caa_masks)
dataset_base_dir = "/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['labels', '.DS_Store', 'images']

Data Augmentation in Progress ...


In [43]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/",'*.png'))

all_images.sort()
all_masks.sort()

cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [44]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(225, 273, 212, 206)

In [61]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/",'*.png'))

In [62]:
all_images.sort()
all_masks.sort()

In [60]:
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        os.remove(image)
        os.remove(mask)
        
    

In [76]:
all_images.sort()
all_masks.sort()

In [77]:
all_masks_img = [x.split("/")[-1] for x in all_masks]

In [78]:
counter = 0
for image in all_images:
    #print(image.split("/")[-1].replace("image","mask"))
    if image.split("/")[-1].replace("image","mask") not in all_masks_img:
        counter=counter+1
        print(image)
        os.remove(image)
print(counter)

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/XE12-010_1_AmyB_1_30926x_157422y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/XE17-004_1_AmyB_1_71704x_156347y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/images/XE19-010_1_AmyB_1_47110x_109293y_image.png
3


In [79]:
all_images_mask = [x.split("/")[-1] for x in all_images]

In [80]:
counter = 0
for mask in all_masks:
    #print(image.split("/")[-1].replace("image","mask"))
    if mask.split("/")[-1].replace("mask","image") not in all_images_mask:
        counter=counter+1
        print(mask)
        os.remove(mask)
print(counter)

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/A_118_0_XE17-039_1_AmyB_1_36877x_104249y_mask.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/XE07-047_1_AmyB_1_29885x_105470y_mask.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/train/labels/XE07-047_1_AmyB_1_54461x_139262y_mask.png
3


In [56]:
all_images = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/images/",'*.png'))
all_masks = glob(os.path.join("/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/labels/",'*.png'))

In [57]:
all_images.sort()
all_masks.sort()
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        os.remove(image)
        os.remove(mask)

/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/images/XE13-007_1_AmyB_1_37037x_114458y_image.png
/gladstone/finkbeiner/steve/work/data/npsad_data/vivek/Datasets/amyb_wsi/val/images/XE13-007_1_AmyB_1_88237x_142106y_image.png


## data augmentations (in runpod)

In [6]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/images/",'*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/labels/",'*.png'))

In [7]:
all_images.sort()
all_masks.sort()

In [29]:
transforms  = A.Compose([A.VerticalFlip(p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.Blur(blur_limit=1,p=0.1),
                            A.OpticalDistortion(p=0.25),
                            A.HueSaturationValue(hue_shift_limit=1, sat_shift_limit=2, val_shift_limit=1, p=0.25),
                            A.RandomRotate90(p=0.5),
                            A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.25)
                        ])


def get_randimages_dataug(total_imgs, image_filenames, label_filenames):
    image_filenames.sort()
    label_filenames.sort()
    random_image_file = []
    random_label_file = []
    for i in range(total_imgs):
        random.seed(i)
        random_image_file.append(random.choice(image_filenames))
        random.seed(i)
        random_label_file.append(random.choice(label_filenames))
    return [random_image_file, random_label_file]

def upsample_dataset(dataset_base_dir, random_img_filenames, rand_label_filenames, variations, transforms, dest_img_folder_name, dest_label_folder_name):
    i = 0
    aug_img_files = []
    aug_mask_files = []
    # random.seed(500)

    # Make dir where tha augmented file will reside
    aug_img_dir = os.path.join(dataset_base_dir, dest_img_folder_name)
    if not os.path.exists(aug_img_dir):
            os.makedirs(aug_img_dir)
            print("Augmented Directory '%s' created" %aug_img_dir)
    
    aug_mask_dir = os.path.join(dataset_base_dir, dest_label_folder_name)
    if not os.path.exists(aug_mask_dir):
            os.makedirs(aug_mask_dir)
            print("Augmented Directory '%s' created" %aug_mask_dir)

    print(os.listdir(dataset_base_dir))
    print("\nData Augmentation in Progress ...")
    total_imgs = len(random_img_filenames)
    
    for i in range(total_imgs):
        # load the image
        img = Image.open(random_img_filenames[i]).convert("RGB")
        img = np.array(img)
        mask = Image.open(rand_label_filenames[i]).convert('P')
        mask = np.array(mask)

        for j in range(variations):
                #transformed = transforms(image=img, mask=mask)
                #transformed_img = transformed["image"]
                #transformed_img = Image.fromarray(transformed_img)
                transformed_img = Image.fromarray(img) 
                #To rename the file with prefix A_
                filename = os.path.basename(random_img_filenames[i])
                filepath = os.path.dirname(random_img_filenames[i])

                #aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                aug_file_name = filename
                new_file = os.path.join(dataset_base_dir,dest_img_folder_name,
                                        aug_file_name)

                transformed_img.save(new_file)
                aug_img_files.append(new_file)

                #transformed_mask = transformed["mask"]
                #transformed_mask = Image.fromarray(transformed_mask)
                transformed_mask = Image.fromarray(mask)  
            
                #To rename the file with prefix A_
                filename = os.path.basename(rand_label_filenames[i])
                filepath = os.path.dirname(rand_label_filenames[i])
                #aug_file_name = "A_" + str(i) + "_" + str(j) + "_" + filename
                aug_file_name = filename
                new_file = os.path.join(dataset_base_dir, dest_label_folder_name,
                                        aug_file_name)
                transformed_mask.save(new_file)
                aug_mask_files.append(new_file)
    return aug_img_files, aug_mask_files

In [10]:
cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [11]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(166, 202, 178, 72)

In [18]:
166 + 202 + 178 + 72

618

In [12]:
aug_value = 100
rand_image_filenames, rand_label_filenames = get_randimages_dataug(aug_value, caa_images, caa_masks)
dataset_base_dir = "/workspace/Projects/Amyb_plaque_detection/Datasets/train"


In [13]:
len(rand_image_filenames)

100

In [14]:
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, rand_image_filenames, rand_label_filenames, 1, transforms, "images", "labels")

['images', '.DS_Store', 'labels']

Data Augmentation in Progress ...


In [15]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/images/",'*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/labels/",'*.png'))
all_images.sort()
all_masks.sort()

In [20]:
len(all_images),  len(all_masks)

(586, 586)

In [22]:
dataset_base_dir = "/workspace/Projects/Amyb_plaque_detection/Datasets/train"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, all_images, all_masks, 5, transforms, "images", "labels")

['images', '.DS_Store', 'labels']

Data Augmentation in Progress ...


In [23]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/images/",'*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/train/labels/",'*.png'))
all_images.sort()
all_masks.sort()

In [ ]:
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if len(found)<=1:
        print(image)
        print(mask)
        os.remove(image)
        os.remove(mask)

In [3]:
## data augmentations (in runpod) VAL

In [10]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/val/images/",'*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/val/labels/",'*.png'))

In [11]:
all_images.sort()
all_masks.sort()

In [12]:
cored_images, cored_masks, diffuse_images, diffuse_masks, cg_images,cg_masks, caa_images, caa_masks  = [],[],[],[],[],[],[],[]
for image, mask in zip(all_images,all_masks):
    found = np.unique(np.array(Image.open(mask)))
    if 50 in found:
        cored_images.append(image)
        cored_masks.append(mask)
    if 100 in found:
        diffuse_images.append(image)
        diffuse_masks.append(mask)
    if 150 in found:
        cg_images.append(image)
        cg_masks.append(mask)
    if 200 in found:
        caa_images.append(image)
        caa_masks.append(mask)

In [8]:
len(cored_images),  len(diffuse_images), len(cg_images), len(caa_images)

(75, 103, 43, 16)

In [13]:
dataset_base_dir = "/workspace/Projects/Amyb_plaque_detection/Datasets/val"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, all_images, all_masks, 5, transforms, "images", "labels")

['images', 'labels']

Data Augmentation in Progress ...


In [14]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/val/images/",'*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/val/labels/",'*.png'))

In [16]:
len(all_images)

1200

In [21]:
all_images = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/test",'*/images/*.png'))
all_masks = glob(os.path.join("/workspace/Projects/Amyb_plaque_detection/Datasets/test",'*/labels/*.png'))

In [22]:
all_images.sort()
all_masks.sort()

In [25]:
dataset_base_dir = "/workspace/Projects/Amyb_plaque_detection/Datasets/test"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, all_images, all_masks, 5, transforms, "images", "labels")

['labels', 'images', 'XE18-066_1_AmyB_1', 'XE07-049_1_AmyB_1', 'XE11-039_1_AmyB_1', 'XE18-003_1_AmyB_1', 'XE19-037_1_AmyB_1', '.DS_Store']

Data Augmentation in Progress ...


In [30]:
dataset_base_dir = "/workspace/Projects/Amyb_plaque_detection/Datasets/test"
aug_img_files, aug_mask_files = upsample_dataset(dataset_base_dir, all_images, all_masks, 1, transforms, "images", "labels")

['labels', 'images', 'XE18-066_1_AmyB_1', 'XE07-049_1_AmyB_1', 'XE11-039_1_AmyB_1', 'XE18-003_1_AmyB_1', 'XE19-037_1_AmyB_1', '.DS_Store']

Data Augmentation in Progress ...
